In [1]:
import pandas as pd
import pickle, json, requests, base64


In [2]:
def get_all_methods():
    df = pd.read_csv("caret_model_infos_detailed.csv")
    methods = df['Method'].apply(str).unique()
    return methods


In [3]:
def get_github_issues_for_caret(state="open"):
    url = "http://api.github.com/repos/topepo/caret/issues?per_page=100&state=" + state
    r = requests.get(url)
    d1 = r.json()
    d = d1
    print("response_headers_0" , r.headers)
    r.close()
    while('Link' in r.headers.keys() and 'rel="next"' in r.headers['Link']):
        # print("response_headers" , r.headers)
        links = r.headers['Link'].split(",")
        print("LINKS" , links)
        if(len(links) == 2):
            next_link = links[0].split(";")[0][1:-1]
        else:
            next_link = links[1].split(";")[0][2:-1]
        r = requests.get(next_link)
        d = r.json()
        print("next_link" , next_link , len(d))
        d1 = d1 + d
    if(isinstance(d1 , dict)):
        d1 = []
    return d1


In [4]:
def parse_file(filename):
    import json
    rows = []
    cols = None
    with open(filename, 'r') as f:
        data = json.load(f)
        for issue in data:
            issue['pull_request'] = None
            # for k in issue.keys():
            #    print("ISSUE_DATA_DETAIL" , k , str(issue[k])[1:50])
            row = [str(issue[k]) for k in sorted(issue.keys())]
            rows.append(row)
            keys = list(sorted(issue.keys()))
            if(cols is None):
                cols = keys
                print(cols)
            else:
                if(keys != cols):
                    print(keys)
                    print(cols)                    
                assert(keys == cols)

    df = pd.DataFrame(rows, columns=cols)
    return df


In [5]:
def get_issue_data_from_github_api():
    issues_open = get_github_issues_for_caret("open")
    issues_closed = get_github_issues_for_caret("closed")
    issues = issues_closed + issues_open

    import json
    with open('github_caret_issues.json', 'w') as outfile:
        json.dump(issues, outfile)
    
# uncomment this to get fresh data !!!!
#get_issue_data_from_github_api()

In [6]:
issues_df = parse_file(filename="github_caret_issues.json")

['assignee', 'assignees', 'author_association', 'body', 'closed_at', 'comments', 'comments_url', 'created_at', 'events_url', 'html_url', 'id', 'labels', 'labels_url', 'locked', 'milestone', 'node_id', 'number', 'pull_request', 'repository_url', 'state', 'title', 'updated_at', 'url', 'user']


In [7]:
issues_df

,assignee,assignees,author_association,body,closed_at,comments,comments_url,created_at,events_url,html_url,...,milestone,node_id,number,pull_request,repository_url,state,title,updated_at,url,user
0,None,[],NONE,- [✓] Start a new R session\r\n- [✓] Install t...,2018-08-31T19:55:06Z,2,https://api.github.com/repos/topepo/caret/issu...,2018-08-31T12:17:51Z,https://api.github.com/repos/topepo/caret/issu...,https://github.com/topepo/caret/issues/936,...,None,MDU6SXNzdWUzNTU5NTc2NTY=,936,None,https://api.github.com/repos/topepo/caret,closed,"Predicting on left-out folds with recipes, rsa...",2018-08-31T19:55:06Z,https://api.github.com/repos/topepo/caret/issu...,"{'login': 'BenjaK', 'id': 4249949, 'node_id': ..."
1,None,[],NONE,I am calculating a performance metric based on...,2018-08-24T20:26:45Z,1,https://api.github.com/repos/topepo/caret/issu...,2018-08-24T16:39:12Z,https://api.github.com/repos/topepo/caret/issu...,https://github.com/topepo/caret/issues/933,...,None,MDU6SXNzdWUzNTM4NTI1ODA=,933,None,https://api.github.com/repos/topepo/caret,closed,Sample size of each resample in the bootstrap,2018-08-24T20:33:05Z,https://api.github.com/repos/topepo/caret/issu...,"{'login': 'dnzmarcio', 'id': 11599265, 'node_i..."
2,None,[],NONE,,2018-08-23T03:14:23Z,0,https://api.github.com/repos/topepo/caret/issu...,2018-08-23T02:19:05Z,https://api.github.com/repos/topepo/caret/issu...,https://github.com/topepo/caret/issues/932,...,None,MDU6SXNzdWUzNTMxOTUxNzc=,932,None,https://api.github.com/repos/topepo/caret,closed,Bayesian Optimization of Machine Learning Models,2018-08-23T03:14:23Z,https://api.github.com/repos/topepo/caret/issu...,"{'login': 'junjiwoo', 'id': 16776755, 'node_id..."
3,None,[],CONTRIBUTOR,Minor fix for #930.,2018-08-22T23:55:43Z,2,https://api.github.com/repos/topepo/caret/issu...,2018-08-22T22:38:15Z,https://api.github.com/repos/topepo/caret/issu...,https://github.com/topepo/caret/pull/931,...,None,MDExOlB1bGxSZXF1ZXN0MjEwMjg0MjA0,931,None,https://api.github.com/repos/topepo/caret,closed,Minor fix for 930,2018-08-22T23:55:43Z,https://api.github.com/repos/topepo/caret/issu...,"{'login': 'hadjipantelis', 'id': 4099384, 'nod..."
4,None,[],NONE,Feature plot sometimes returns a coercion erro...,2018-08-17T14:22:00Z,2,https://api.github.com/repos/topepo/caret/issu...,2018-08-13T10:02:53Z,https://api.github.com/repos/topepo/caret/issu...,https://github.com/topepo/caret/issues/925,...,None,MDU6SXNzdWUzNDk5NjQ3Mzk=,925,None,https://api.github.com/repos/topepo/caret,closed,FeaturePlot tibble dependence,2018-08-17T14:22:00Z,https://api.github.com/repos/topepo/caret/issu...,"{'login': '0Matt', 'id': 11705852, 'node_id': ..."
5,None,[],CONTRIBUTOR,Should `caret` and/or the parallel backend avo...,2018-08-17T18:23:22Z,4,https://api.github.com/repos/topepo/caret/issu...,2018-08-12T21:27:33Z,https://api.github.com/repos/topepo/caret/issu...,https://github.com/topepo/caret/issues/924,...,None,MDU6SXNzdWUzNDk4NDU0NTA=,924,None,https://api.github.com/repos/topepo/caret,closed,Nested parallelism with xgboost,2018-08-17T18:23:22Z,https://api.github.com/repos/topepo/caret/issu...,"{'login': 'adatum', 'id': 9773655, 'node_id': ..."
6,None,[],NONE,CODE:\r\nlibrary(caret); library(kernlab); dat...,2018-08-17T14:37:18Z,1,https://api.github.com/repos/topepo/caret/issu...,2018-08-10T15:13:53Z,https://api.github.com/repos/topepo/caret/issu...,https://github.com/topepo/caret/issues/923,...,None,MDU6SXNzdWUzNDk1NTQ0MTU=,923,None,https://api.github.com/repos/topepo/caret,closed,undefined columns selected,2018-08-17T14:37:18Z,https://api.github.com/repos/topepo/caret/issu...,"{'login': 'pandeyvineet', 'id': 39806528, 'nod..."
7,None,[],CONTRIBUTOR,This regarding issues #890 and #902. Some mino...,2018-08-17T14:45:12Z,1,https://api.github.com/repos/topepo/caret/issu...,2018-07-26T22:53:54Z,https://api.github.com/repos/topepo/caret/issu...,https://github.com/topepo/caret/pull/921,...,None,MDExOlB1bGxSZXF1ZXN0MjA0Mjg4MTc1,921,None,https://api.github.com/repos/top

In [8]:
def get_method_popularity(method):
    issue_text_column = issues_df['title'] + issues_df['body']
    contains_method = issue_text_column.apply(lambda x : 1 if (method in x) else 0)
    return contains_method.sum()
    

In [9]:
methods = get_all_methods()
popularity = {}
for method in methods:
    score = get_method_popularity(method)
    popularity[method] = score
    print("METHOD_POPULARTIY" , method, score)

METHOD_POPULARTIY AdaBag 1
METHOD_POPULARTIY AdaBoost.M1 1
METHOD_POPULARTIY adaboost 1
METHOD_POPULARTIY ada 29
METHOD_POPULARTIY amdai 0
METHOD_POPULARTIY ANFIS 1
METHOD_POPULARTIY avNNet 7
METHOD_POPULARTIY awnb 1
METHOD_POPULARTIY awtan 1
METHOD_POPULARTIY bagEarthGCV 1
METHOD_POPULARTIY bagEarth 9
METHOD_POPULARTIY bagFDAGCV 0
METHOD_POPULARTIY bagFDA 2
METHOD_POPULARTIY bag 30
METHOD_POPULARTIY bam 1
METHOD_POPULARTIY bartMachine 3
METHOD_POPULARTIY bayesglm 3
METHOD_POPULARTIY binda 3
METHOD_POPULARTIY blackboost 1
METHOD_POPULARTIY blassoAveraged 0
METHOD_POPULARTIY blasso 0
METHOD_POPULARTIY bridge 0
METHOD_POPULARTIY brnn 2
METHOD_POPULARTIY BstLm 2
METHOD_POPULARTIY bstSm 1
METHOD_POPULARTIY bstTree 0
METHOD_POPULARTIY C5.0Cost 0
METHOD_POPULARTIY C5.0 7
METHOD_POPULARTIY C5.0Rules 0
METHOD_POPULARTIY C5.0Tree 0
METHOD_POPULARTIY cforest 3
METHOD_POPULARTIY chaid 0
METHOD_POPULARTIY CSimca 0
METHOD_POPULARTIY ctree2 1
METHOD_POPULARTIY ctree 3
METHOD_POPULARTIY cubist 2
METH

In [10]:
import operator
sorted_methods = sorted(popularity.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_methods)


[('rf', 252), ('lm', 239), ('nnet', 169), ('tan', 134), ('glm', 102), ('rpart', 83), ('pda', 76), ('dda', 68), ('glmnet', 52), ('gbm', 51), ('ppr', 48), ('ranger', 42), ('gam', 39), ('knn', 37), ('xgbTree', 37), ('bag', 30), ('ada', 29), ('pls', 28), ('nb', 23), ('svmRadial', 21), ('earth', 20), ('mlp', 20), ('nan', 18), ('svmLinear', 16), ('lda', 15), ('lars', 11), ('bagEarth', 9), ('mxnet', 9), ('fda', 8), ('pam', 8), ('rbf', 8), ('avNNet', 7), ('C5.0', 7), ('enet', 7), ('lmStepAIC', 7), ('dnn', 6), ('icr', 6), ('J48', 6), ('nnls', 6), ('Rborist', 6), ('RRF', 6), ('deepboost', 5), ('lasso', 5), ('mda', 5), ('multinom', 5), ('sda', 5), ('wsrf', 5), ('evtree', 4), ('glmboost', 4), ('mlpML', 4), ('monmlp', 4), ('mxnetAdam', 4), ('neuralnet', 4), ('rda', 4), ('ridge', 4), ('sparseLDA', 4), ('spls', 4), ('svmLinear2', 4), ('xgbLinear', 4), ('bartMachine', 3), ('bayesglm', 3), ('binda', 3), ('cforest', 3), ('ctree', 3), ('extraTrees', 3), ('GFS.FR.MOGUL', 3), ('glmStepAIC', 3), ('mlpWeight